## ETL process

In [98]:
import glob                          # this module helps in selecting files
import pandas as pd                  # this module helps in processing CSV files
import xml.etree.ElementTree as ET   # this module helps in processing XML files.
import urllib.request                # to download the data set
from datetime import datetime

In [55]:
urllib.request.urlretrieve('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip', 'dealership_data.zip')

('dealership_data.zip', <http.client.HTTPMessage at 0x1aca1d3b8e0>)

In [99]:
#setting paths
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

In [100]:
#csv extraction
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [101]:
#json extraction
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

In [102]:
#xml Extraction
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for node in root:
        car_model = node.find("car_model").text
        year_of_manufacture = int(node.find("year_of_manufacture").text)
        price = float(node.find("price").text)
        fuel = node.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe

#extract_from_xml('dealership_data/used_car_prices3.xml')


In [103]:
#debugging
for csvfile in glob.glob("dealership_data/*.csv"):
    print(csvfile)

dealership_data\used_car_prices1.csv
dealership_data\used_car_prices2.csv
dealership_data\used_car_prices3.csv


In [104]:
#debugging and appending
extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(pd.read_csv(csvfile), ignore_index=True)
print(extracted_data)

        car_model year_of_manufacture         price    fuel
0            ritz                2014   5000.000000  Petrol
1             sx4                2013   7089.552239  Diesel
2            ciaz                2017  10820.895522  Petrol
3         wagon r                2011   4253.731343  Petrol
4           swift                2014   6865.671642  Diesel
5   vitara brezza                2018  13805.970149  Diesel
6            ciaz                2015  10074.626866  Petrol
7         s cross                2015   9701.492537  Diesel
8            ciaz                2016  13059.701493  Diesel
9            ciaz                2015  11119.402985  Diesel
10       alto 800                2017   4253.731343  Petrol
11           ciaz                2015  10223.880597  Diesel
12           ciaz                2015  11194.029851  Petrol
13         ertiga                2015   9104.477612  Petrol
14          dzire                2009   3358.208955  Petrol
15         ertiga                2016  1

In [105]:
#Extract Function
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

In [106]:
#transform function
#Round the `price` columns to 2 decimal places
def transform(data):
    data['price'] = round(data.price, 2)
    return data

In [107]:
#load
def load(targetfile, data_to_be_loaded):
    data_to_be_loaded.to_csv(targetfile)

In [108]:
#logging
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

##### Running ETL process

In [109]:
#Log that you have started the ETL process
log("ETL Job Started")

#Log that you have started the Extract step
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

#Log that you have started the Transform step
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

#Log that you have started the Load step
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

#Log that you have completed the ETL process
log("ETL Job Ended")

In [110]:
# reading transformed_data file
# index_col is to remove the un-named index
df = pd.read_csv("dealership_transformed_data.csv", index_col=[0])
print(df) 

        car_model  year_of_manufacture     price    fuel
0            ritz                 2014   5000.00  Petrol
1             sx4                 2013   7089.55  Diesel
2            ciaz                 2017  10820.90  Petrol
3         wagon r                 2011   4253.73  Petrol
4           swift                 2014   6865.67  Diesel
..            ...                  ...       ...     ...
85          camry                 2006   3731.34  Petrol
86   land cruiser                 2010  52238.81  Diesel
87  corolla altis                 2012   8805.97  Petrol
88     etios liva                 2013   5149.25  Petrol
89        etios g                 2014   7089.55  Petrol

[90 rows x 4 columns]
